In [58]:
def scriptExec(printador=False):
    #ls_turmas = {'T1', 'T2', 'T3'}
    ls_turmas = {'T1'}
    ls_listas = {'Lista01','Lista02','Lista03','Lista04','Lista05'}
    
    for turma in sorted(ls_turmas):
        for lista in sorted(ls_listas):
            main(turma, lista, printador)
        break
            
#scriptExec()

In [4]:
import spacy
import math
import numpy as np
import pandas as pd
import seaborn as sb
import re
import os

%run PY_preprocessing.ipynb
%run PY_extrator.ipynb
%run PY_machine_learning.ipynb
%run PY_utilities.ipynb

### PATH SETTINGS
root_input = "db-input"
root_output = "db-output"
root_tree = "images/"
output_file = "csv-saida-extrator.csv"

### CLUSTERING SETTINGS
metric = "euclidean"
method = "average"

### INPUT SETTINGS
ls_types = {'void', 'int', 'long', 'float', 'double', 'char'}
datatype = "(void|int|float|double|char)"

def main(turma="T1", lista="Lista01", printador=False, num_clusters=4):
    clearOutputCSV(output_file)
    # -------------------------------------------------------------------------------
    # (ls_dataset): armazena um dicionário com as features e seus valores 
    ls_dataset = []
            
    # files settings
    folder_entrada, output_folder, output_tree, output_cluster = outputFilesIdentifier(root_input, root_output, turma, lista, root_tree)

    # create output folder if do not exist
    mkdirOutputFolder(output_folder)
            
    # join every source code solutions
    ls_todos_codigos, ls_name_files = join_tasks(folder_entrada, output_folder, True)

    # -------------------------------------------------------------------------------
    # OUTPUT: ls_todos_codigos: [[[cod1-joao], [cod2-joao]], [[cod1-maria], [cod2-maria]]]
    for id_aluno, (codigos_aluno, name_codes) in enumerate(zip(ls_todos_codigos, ls_name_files)):
        
        # O id de aluno inicia em 1
        id_aluno += 1
                        
        if lista == "Lista01":                        
            features01 = set_list01()
                    
        if lista == "Lista02":
            features02 = set_list02()

        if lista == "Lista03":            
            features03 = set_list03()

        if lista == "Lista04":                
            features04 = set_list04()
                    
        if lista == "Lista05":                
            features05 = set_list05()
 
        ##-------------------------------------------------------------------------------                
        if (printador):
            get_printador_cab(id_aluno)
            print("n. of source codes: {}".format(len(codigos_aluno)))
                    
        ##-------------------------------------------------------------------------------
        ## EXTRATOR
        for id_cod, (codigo, name_code) in enumerate(zip(codigos_aluno, name_codes)):
            
            codigo = removeBlockComment(codigo)
            codigo = removeLineComment(codigo)

            ##-------------------------------------------------------------------------------                     
            ls_signatures, ls_flags = get_signatures(codigo, ls_types)
            ls_functions = get_functions(codigo)
            ls_calls = get_calls(codigo)
            ls_func_call = get_func_call(codigo, ls_flags)
            ##-------------------------------------------------------------------------------    

            #if lista == "Lista01" or lista == "Lista02":
            if lista == "Lista01":                
                features01 = get_list01(features01, codigo, ls_calls)
                        
                ls_libs = get_calls_libs(codigo, name_codes)
                if not ls_libs == "":
                    for (c, n) in zip(codigos_aluno, name_codes):
                        if ls_libs in n:
                            ls_signatures, ls_flags = get_signatures(c, ls_types)
                            features01['NOC'] += len(get_calls(codigo))
                            #print("mudou noc: {}".format(len(get_calls(codigo, ls_signatures))))
                            break
                    
            if lista == "Lista02":
                features02 = get_list02(features02, codigo, ls_functions, ls_calls, ls_func_call)

            if lista == "Lista03":
                features03 = get_list03(features03, codigo)                    

            if lista == "Lista04":
                features04 = get_list04(features04, codigo, ls_functions) 
                        
            if lista == "Lista05":
                features05 = get_list05(features05, codigo) 
                
        ##-------------------------------------------------------------------------------
        dict_list = {}
                                
        if lista == "Lista01":
            dict_list.update(features01)

        if lista == "Lista02":
            dict_list.update(features02)
                                
        if lista == "Lista03":
            dict_list.update(features03)

        if lista == "Lista04":
            dict_list.update(features04)
                    
        if lista == "Lista05":
            dict_list.update(features05)                    

        ##-------------------------------------------------------------------------------                
        if (printador):
            get_printador(dict_list)                    
     
        ls_dataset.append(dict_list)
                   
    # -------------------------------------------------------------------------------        
    # EXTRATOR        
    df = pd.DataFrame(ls_dataset)
    features_cols = df.keys()
            
    # students id (eg.: 101, 201 or 301)
    start_stud = studentsIdentifier(turma)
            
    # get clustering
    df, dfx, c = agrupador(df, features_cols, start_stud, output_cluster, num_clusters, metric, method)            

    # columns name (eg.: NF, FLC, NP, etc)
    column_names = columnsIdentifier(lista)
    print("\n>>> EXTRACTED FEATURES: \n{}".format(column_names))
            
    # write CSV results
    writeHierarchicalCSV(df, column_names, output_file, dfx)
        
    return (ls_dataset)



<Figure size 720x216 with 0 Axes>

In [1]:
ls_dataset = main("T1", "Lista03")